## PINEAPPLER Secret Code


In [645]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [646]:
import os 

os.getcwd()

'/Users/remyzhang/Documents/CIVL498A/Final_project_rezoning'

In [647]:
df = pd.read_csv("./TorontoRezoningDataset3.csv")
df.head(10)



,Name of the Tower,Status,Use,Address,City,Height (ft),# of Floors,Proposed Date,Rezoning Authority,Type Of Approval,OMB case #,Rezoning Approval,Rezoning Duration,Start,Completion,Major Arterial Road?,Distance to Nearest School (m)
0,The one,Under Construction,residential / retail,1- 11 Bloor Street west & 768-784 Yonge Street,Toronto,"1,117",84,2015-03-16,OMB,NaN,NaN,2016-07-12,1.3,2016,2019.0,Yes,150
1,Mirvish+Gehry - West Tower,Proposed,residential / office,274 - 322 King Street West,Toronto,998,92,2012-11-09,OMB,Hearing,PL130629,2014-07-08,1.7,NaN,NaN,Yes,300
2,One Bloor Street East,Built,residential / retail,"1 and 23 Bloor Street East, 14 Hayden Street a...",Toronto,844,75,2007-11-09,City Council,NaN,NaN,2008-10-29,1.0,2011,2016.0,Yes,200
3,Mirvish+Gehry - East Tower,proposed,residential / office,274 - 322 King Street west,Toronto,899,82,2012-11-09,OMB,Hearing,PL130629,2014-07-08,1.7,NaN,NaN,Yes,400
4,One Yonge Street Tower 2,proposed,residential,1 — 7 Yonge Street,Toronto,853,80,2013-03-06,OMB,Settlement Hearing,PL140008,NaN,NaN,NaN,NaN,Yes,100
5,37 Yorkville Tower A,proposed,residential,"27-37 Yorkville Avenue and 26-32, 50 Cumberlan...",Toronto,779,69,2012-06-12,City Council,NaN,NaN,2014-08-25,2.2,NaN,NaN,Yes,250
6,ICE 2 Condominiums at York Centre,Built,residential,14 York Street,Toronto,768,67,2008-01-04,City Council,NaN,NaN,2009-04-01,1.2,2011,2015.0,Yes,300
7,Harbour Plaza Residences East,Under Construction,residential,90 Harbour Street,Toronto,764,66,2011-10-19,OMB,Hearing,PL151171,2012-11-27,1.1,2013,2017.0,Yes,100
8,50 Bloor West,Proposed,residential / office / retail,50 Bloor Street West,Toronto,754,71,2012-03-22,City Council,NaN,NaN,2014-08-25,2.4,NaN,NaN,Yes,150
9,Eau de Soleil Sky Tower,Under Construction,residential,2183 Lake Shore Blvd West,Toronto,749,66,2011-10-28,OMB,Open,MM150071,2013-12-01,2.1,2015,2018.0,Yes,500


In [648]:
# Drop the "City" beacuse all share the same city 
df = df.drop(columns = ['City'])
df = df.drop(columns = ["Type Of Approval"])
df = df.drop(columns = ["Name of the Tower"])
# The Start and Completion Dates refer the construction dates after the rezoning process has finished, so we can disregard them.
df = df.drop(columns = ["Start" , "Completion"])
df = df.drop(columns = ["Address"])
df = df.drop(columns = ["Rezoning Approval"])
df = df.drop(columns = ["Rezoning Duration"])



In [649]:
df["Rezoned Status"] = np.where(df["OMB case #"].notnull(), 0,
                                np.where(df["Status"].isin(["Under Construction", "Built"]), 1, np.nan))
                            
                            

In [650]:
df.head(10)


,Status,Use,Height (ft),# of Floors,Proposed Date,Rezoning Authority,OMB case #,Major Arterial Road?,Distance to Nearest School (m),Rezoned Status
0,Under Construction,residential / retail,"1,117",84,2015-03-16,OMB,NaN,Yes,150,1.0
1,Proposed,residential / office,998,92,2012-11-09,OMB,PL130629,Yes,300,0.0
2,Built,residential / retail,844,75,2007-11-09,City Council,NaN,Yes,200,1.0
3,proposed,residential / office,899,82,2012-11-09,OMB,PL130629,Yes,400,0.0
4,proposed,residential,853,80,2013-03-06,OMB,PL140008,Yes,100,0.0
5,proposed,residential,779,69,2012-06-12,City Council,NaN,Yes,250,NaN
6,Built,residential,768,67,2008-01-04,City Council,NaN,Yes,300,1.0
7,Under Construction,residential,764,66,2011-10-19,OMB,PL151171,Yes,100,0.0
8,Proposed,residential / office / retail,754,71,2012-03-22,City Council,NaN,Yes,150,NaN
9,Under Construction,residential,749,66,2011-10-28,OMB,MM150071,Yes,500,0.0


In [651]:
df_new = df.drop(df[(df['Status'].str.lower() == "proposed") & (df['OMB case #'].isnull())].index)
# drop all cases where BOTH the "Status" column is "proposed" and the "OMB case #" column is NaN
# we drop these values because the result of the application is unknown.


In [652]:
df_new = df_new.drop(columns = ["OMB case #", "Status"])


df_new.head(10)
num_columns = df.shape[0]
print(num_columns)
df_new.info()

174
<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 171
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Use                             151 non-null    object 
 1   Height (ft)                     151 non-null    object 
 2   # of Floors                     151 non-null    int64  
 3   Proposed Date                   151 non-null    object 
 4   Rezoning Authority              151 non-null    object 
 5   Major Arterial Road?            151 non-null    object 
 6   Distance to Nearest School (m)  151 non-null    int64  
 7   Rezoned Status                  151 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 10.6+ KB


## Treating Missing or Incorrect Values

In [653]:
#missing_height = df[df["Height (ft)"].isnull()]
#print(missing_height)
#print("\n")

#missing_height = df_new[df_new["Height (ft)"].isnull()]
#print(missing_height)


#df_new.loc[133, "Height (ft)"] = 318

#print(df_new.loc[133])

# The tower: "Studio on Richmond" has its height missing, a Google Search tell us it is 318 ft tall
df_new["Height (ft)"] = df_new["Height (ft)"].str.replace(",", "").astype(float)

## Encoding

Here we encode the "Use" feature as it has multiple unique options. We use Binary encoding because there is some inherent structure to the five unique options, "Education", "hotel", "office", "residential", "retail"

In [654]:
from sklearn.preprocessing import OneHotEncoder

# Split "Use" column into multiple binary columns
use_dummies = df_new["Use"].str.get_dummies(sep=" / ")      # Note in the data, the unique options within "Use" are split by a " / "

# Concat the newly created binary columns onto a new dataframe
df_encoded = pd.concat([df_new, use_dummies], axis=1)

# Drop the original "Use" column
df_encoded.drop("Use", axis=1, inplace=True)
df_encoded.head(10)
print("New dataframe size:", df_encoded.shape)
df_encoded.info()


New dataframe size: (151, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 171
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Height (ft)                     151 non-null    float64
 1   # of Floors                     151 non-null    int64  
 2   Proposed Date                   151 non-null    object 
 3   Rezoning Authority              151 non-null    object 
 4   Major Arterial Road?            151 non-null    object 
 5   Distance to Nearest School (m)  151 non-null    int64  
 6   Rezoned Status                  151 non-null    float64
 7   education                       151 non-null    int64  
 8   hotel                           151 non-null    int64  
 9   office                          151 non-null    int64  
 10  residential                     151 non-null    int64  
 11  retail                          151 non-null    int64  
dtypes: flo

Here we encode the Rezoning Authority feature into its uniqule options "OMB" and "City Council", representing which body of government the rezoning application is approved by.

In [655]:
# # create an instance of the OneHotEncoder
# encoder = OneHotEncoder(sparse=False)

# # fit and transform the "Rezoning Authority" column
# encoded = encoder.fit_transform(df_encoded[["Rezoning Authority"]])

# # create a new DataFrame with the encoded column(s)
#df_encoded2 = df_encoded.copy()
# df_encoded2 = pd.concat([df_encoded, pd.DataFrame(encoded)], axis=1)

# df_encoded2.drop("Rezoning Authority", axis=1, inplace=True)

# df_encoded2.info()

In [ ]:
df_encoded2 = df_encoded.copy()
df_encoded2.drop("Rezoning Authority", axis=1, inplace=True)

In [656]:
# Now we encode "Proposed Date". The month and date of the proposal have less effect on the Long term affects of rezoning policy change.
# By breaking down Proposed Date to Propsed Year. We can have a big picture idea if rezoning policies changes through time have an effect.

df_encoded3 = df_encoded2.copy()
df_encoded3["Proposed Date"] = pd.to_datetime(df["Proposed Date"], format="%Y-%m-%d")

df_encoded3["Proposed Year"] = df_encoded3["Proposed Date"].dt.year
df_encoded3.drop("Proposed Date", axis=1, inplace=True)

#df_encoded3.info()
#print(df['Proposed Date'].isnull().sum())
#print(df_encoded3[df_encoded3['Proposed Year'].isnull()])

print("Original dataframe size:", df_encoded.shape)
print("New dataframe size:", df_encoded3.shape)


Original dataframe size: (151, 12)
New dataframe size: (151, 11)


In [657]:
df_encoded4 = df_encoded3.copy()
df_econded4 = df_encoded4['Major Arterial Road?'] = df['Major Arterial Road?'].replace({'Yes': 1.0, 'No': 0.0})

df_encoded4.head()


,Height (ft),# of Floors,Major Arterial Road?,Distance to Nearest School (m),Rezoned Status,education,hotel,office,residential,retail,Proposed Year
0,1117.0,84,1.0,150,1.0,0,0,0,1,1,2015
1,998.0,92,1.0,300,0.0,0,0,1,1,0,2012
2,844.0,75,1.0,200,1.0,0,0,0,1,1,2007
3,899.0,82,1.0,400,0.0,0,0,1,1,0,2012
4,853.0,80,1.0,100,0.0,0,0,0,1,0,2013


In [658]:
df_encoded4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 171
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Height (ft)                     151 non-null    float64
 1   # of Floors                     151 non-null    int64  
 2   Major Arterial Road?            151 non-null    float64
 3   Distance to Nearest School (m)  151 non-null    int64  
 4   Rezoned Status                  151 non-null    float64
 5   education                       151 non-null    int64  
 6   hotel                           151 non-null    int64  
 7   office                          151 non-null    int64  
 8   residential                     151 non-null    int64  
 9   retail                          151 non-null    int64  
 10  Proposed Year                   151 non-null    int64  
dtypes: float64(3), int64(8)
memory usage: 14.2 KB


## Model Training

In [659]:
df_train = df_encoded4.copy()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 171
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Height (ft)                     151 non-null    float64
 1   # of Floors                     151 non-null    int64  
 2   Major Arterial Road?            151 non-null    float64
 3   Distance to Nearest School (m)  151 non-null    int64  
 4   Rezoned Status                  151 non-null    float64
 5   education                       151 non-null    int64  
 6   hotel                           151 non-null    int64  
 7   office                          151 non-null    int64  
 8   residential                     151 non-null    int64  
 9   retail                          151 non-null    int64  
 10  Proposed Year                   151 non-null    int64  
dtypes: float64(3), int64(8)
memory usage: 14.2 KB


In [665]:
X = df_train.iloc[:, :4].join(df_train.iloc[:, 5:])
X.info()
y = df_train['Rezoned Status']

y.info()


# Split data 80 train 20 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 171
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Height (ft)                     151 non-null    float64
 1   # of Floors                     151 non-null    int64  
 2   Major Arterial Road?            151 non-null    float64
 3   Distance to Nearest School (m)  151 non-null    int64  
 4   education                       151 non-null    int64  
 5   hotel                           151 non-null    int64  
 6   office                          151 non-null    int64  
 7   residential                     151 non-null    int64  
 8   retail                          151 non-null    int64  
 9   Proposed Year                   151 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 17.0 KB
<class 'pandas.core.series.Series'>
Int64Index: 151 entries, 0 to 171
Series name: Rezoned Status
Non-Null Count

#### Logistic Regression Classification Model

In [666]:
from sklearn.linear_model import LogisticRegression

# Create a logistic regression object
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Predict the class labels for the test set
y_pred = lr.predict(X_test)

In [668]:
# Calculate the accuracy score of the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

print('Logistic Regression Model Accuracy: {:.2f}%'.format(accuracy*100))

Logistic Regression Model Accuracy: 64.52%


#### Decision Tree Model

In [669]:
from sklearn.tree import DecisionTreeClassifier

# Create a decision tree object
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

# Predict the class labels for the test set
y_pred = dt.predict(X_test)

In [670]:
accuracy = accuracy_score(y_test, y_pred)

print('Decision Tree Model Accuracy: {:.2f}%'.format(accuracy*100))

Logistic Regression Model Accuracy: 48.39%


#### Random Forest Model

In [672]:
from sklearn.ensemble import RandomForestClassifier

# create a random forest classifier object
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# fit the model to the training data
rf.fit(X_train, y_train)

# make predictions on the test set
y_pred = rf.predict(X_test)

In [674]:
accuracy = rf.score(X_test, y_test)
print("Random Forest Accuracy: {:.2f}%".format(accuracy * 100))

Random Forest Accuracy: 51.61%


#### SVM Model

In [675]:
from sklearn.svm import SVC

# Create a SVM object and rename it to svm
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)

# Make predictions using the SVM model
y_pred = svm.predict(X_test)

In [678]:
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy: {:.2f}%".format(accuracy * 100))

SVM Accuracy: 58.06%
